In [1]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
from tqdm.autonotebook import tqdm
from sklearn.datasets import fetch_olivetti_faces
from gpzoo.gp_stuff import NSF, NSF_RBF

/var/folders/k0/dnsdn2ws5vl0xhxl_60p6pcr0000gq/T/ipykernel_73859/1508153585.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [3]:
data, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True, random_state=97)

In [4]:
Y = (data)*255
Y = Y.astype(int)
Y = torch.tensor(Y)

In [5]:
idx = torch.linspace(-32, 32, 64, dtype=torch.float)
X = torch.cartesian_prod(-idx, idx).flip(dims=[1])
N = X.shape[0]

In [6]:
L =10
M = 32*32
kernel = NSF_RBF(L=L, lengthscale=2)
model = NSF(X=X, y=Y, kernel=kernel, M=M, L=L, jitter=1e-2)
idz = torch.multinomial(torch.ones(X.shape[0]), num_samples=M, replacement=False)
model.svgp.Z = nn.Parameter(X[idz])

model.to(device)

NSF(
  (kernel): NSF_RBF()
  (svgp): SVGP(
    (kernel): NSF_RBF()
  )
)

In [7]:
X_train = X.to(device)
Y_train = Y.to(device)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
losses = []
for it in tqdm(range(1000)):
            
    optimizer.zero_grad()
    pY, qF, qU, pU = model(X_train, E=20)


    ELBO = (pY.log_prob(Y_train)).mean(axis=0).sum()

    ELBO -= torch.sum(distributions.kl_divergence(qU, pU))


    loss = -ELBO
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
plt.plot(losses)

In [ ]:
plt.figure(figsize=(10, L*2//5))
for i in range(L):
    plt.subplot(L//5, 5, i+1)
    loading = torch.exp(qF.mean[i]).detach().cpu().numpy()
    plt.scatter(X[:, 0].detach().cpu().numpy(), X[:,1].detach().cpu().numpy(), c=loading, cmap='gray',  vmin=0, vmax=10, alpha=0.9, s=2)

    plt.axis('off')